# Implementering af et dyreekspertsystem

Et eksempel fra [AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners).

I dette eksempel vil vi implementere et simpelt vidensbaseret system til at bestemme et dyr baseret på nogle fysiske karakteristika. Systemet kan repræsenteres ved følgende AND-OR træ (dette er en del af det komplette træ, vi kan nemt tilføje flere regler):

![](../../../../../../translated_images/da/AND-OR-Tree.5592d2c70187f283.webp)


## Vores egen ekspertsystem-shell med baglæns inferens

Lad os prøve at definere et enkelt sprog til vidensrepræsentation baseret på produktionsregler. Vi vil bruge Python-klasser som nøgleord til at definere regler. Der vil i det væsentlige være 3 typer klasser:
* `Ask` repræsenterer et spørgsmål, der skal stilles til brugeren. Det indeholder sættet af mulige svar.
* `If` repræsenterer en regel, og det er bare en syntaktisk sukker til at gemme indholdet af reglen
* `AND`/`OR` er klasser til at repræsentere AND/OR-grene af træet. De gemmer blot listen af argumenter indeni. For at forenkle koden er al funktionalitet defineret i forældrekassen `Content`


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

I vores system ville arbejdshukommelsen indeholde listen over **fakta** som **attribut-værdi-par**. Vidensbasen kan defineres som et stort opslagsværk, der forbinder handlinger (nye fakta, der skal indsættes i arbejdshukommelsen) med betingelser, udtrykt som OG-ELLER-udtryk. Desuden kan nogle fakta `Ask`es.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

For at udføre baglæns inferens, vil vi definere klassen `Knowledgebase`. Den vil indeholde:
* Arbejdende `memory` - en ordbog, der forbinder attributter med værdier
* Knowledgebase `rules` i det format, der er defineret ovenfor

To hovedmetoder er:
* `get` for at opnå værdien af en attribut, og udføre inferens om nødvendigt. For eksempel, `get('color')` ville hente værdien af et farveslot (den vil spørge om nødvendigt og gemme værdien til senere brug i arbejdshukommelsen). Hvis vi spørger `get('color:blue')`, vil den spørge efter en farve og derefter returnere `y`/`n` værdi alt efter farven.
* `eval` udfører den faktiske inferens, dvs. gennemgår AND/OR-træet, evaluerer delmål osv.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

Lad os nu definere vores dyrevidensbase og udføre konsultationen. Bemærk, at dette opkald vil stille dig spørgsmål. Du kan svare ved at skrive `y`/`n` for ja-nej spørgsmål, eller ved at angive tal (0..N) for spørgsmål med længere multiple-choice svar.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## Brug af Experta til Fremadrettet Slutning

I det næste eksempel vil vi forsøge at implementere fremadrettet slutning ved hjælp af et af bibliotekerne til videnrepræsentation, [Experta](https://github.com/nilp0inter/experta). **Experta** er et bibliotek til at skabe fremadrettede slutningssystemer i Python, som er designet til at ligne det klassiske gamle system [CLIPS](http://www.clipsrules.net/index.html). 

Vi kunne også have implementeret fremadrettet kædning selv uden mange problemer, men naive implementeringer er normalt ikke særlig effektive. For mere effektiv regelmatchning bruges en speciel algoritme [Rete](https://en.wikipedia.org/wiki/Rete_algorithm).


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/nilp0inter/experta

  Cloning https://github.com/nilp0inter/experta to /tmp/pip-req-build-7qurtwk3
  Running command git clone --filter=blob:none --quiet https://github.com/nilp0inter/experta /tmp/pip-req-build-7qurtwk3
  Resolved https://github.com/nilp0inter/experta to commit c6d5834b123861f5ae09e7d07027dc98bec58741
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for experta: filename=experta-1.9.5.dev1-py3-none-any.whl size=34804 sha256=888c459512a5e713f4b674caa9a0f96cfdf07ec0d6eb56cc318ce0653d218014
  Stored in directory: /tmp/pip-ephem-wheel-cache-1eeii9zy/wheels/3d/e8/bb/22d7956359603fa8dd679aa09f5b8efb3f29991c3986fdc787
Successfully built experta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [experta]


In [13]:
from experta import *
#import experta

Vi vil definere vores system som en klasse, der underklasser `KnowledgeEngine`. Hver regel defineres af en separat funktion med `@Rule` annotation, som angiver, hvornår reglen skal aktiveres. Inde i reglen kan vi tilføje nye fakta ved hjælp af `declare` funktionen, og tilføjelsen af disse fakta vil resultere i, at nogle flere regler bliver kaldt af fremadrettet inferensmotor.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

Når vi har defineret en videnbase, fylder vi vores arbejdshukommelse med nogle indledende fakta og kalder derefter `run()` metoden for at udføre inferensen. Du kan se som resultat, at nye udledte fakta bliver tilføjet til arbejdshukommelsen, inklusive det endelige faktum om dyret (hvis vi har sat alle de indledende fakta korrekt op).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Ansvarsfraskrivelse**:
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på nøjagtighed, bedes du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det oprindelige dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi påtager os intet ansvar for misforståelser eller fejltolkninger, der måtte opstå som følge af brugen af denne oversættelse.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
